## Backtesting 
### Evaluación con Datos de Ejecuciones (tick data)

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with open('../data/COL_tick_data.pkl','rb') as f:
    tick_df = pickle.load(f)

In [ ]:
from mplfinance.original_flavor import candlestick2_ohlc
import mplfinance as mpf

In [ ]:
tick_df

In [ ]:
open_price = tick_df.precio.resample('5min').first()
open_price

In [ ]:
open_price.plot()

### Construcción de Velas 
Con la función resample podemos construir directamente la información OHLCV
sobre la resolución que nos interese

In [ ]:
tick_resolution = '5min'

In [ ]:
open_price = tick_df.precio.resample(tick_resolution).first()
close_price = tick_df.precio.resample(tick_resolution).last()
max_price = tick_df.precio.resample(tick_resolution).max()
min_price = tick_df.precio.resample(tick_resolution).min()
vol_agg = tick_df.volumen.resample(tick_resolution).sum()
bars = pd.concat([open_price, max_price, min_price, close_price, vol_agg], axis=1)
bars.columns = ['open','high','low','close','vol']

In [ ]:
bars

In [ ]:
mpf.plot(bars, type='candle', style='yahoo')

___

### Volumen 

In [ ]:
def get_bars(tick_resolution):
    open_price = tick_df.precio.resample(tick_resolution).first()
    close_price = tick_df.precio.resample(tick_resolution).last()
    max_price = tick_df.precio.resample(tick_resolution).max()
    min_price = tick_df.precio.resample(tick_resolution).min()
    vol_agg = tick_df.volumen.resample(tick_resolution).sum()
    bars = pd.concat([open_price, max_price, min_price, close_price, vol_agg], axis=1)
    bars.columns = ['open','high','low','close','vol']
    return bars

In [ ]:
bars2 = get_bars('30min')
bars2.vol.plot.bar()

### Estimación órdenes

In [ ]:
order_datetime = '2018-01-03 15:00'

In [ ]:
win_trade = tick_df.loc[order_datetime:]
agg_vol = win_trade.volumen.cumsum()
win_trade

In [ ]:
agg_vol

In [ ]:
def get_win_trade(tick_df, timestamp, order_size, vol_pct):
    win_trade = tick_df.loc[order_datetime:].copy()
    agg_vol = win_trade.volumen.cumsum()
    frame = agg_vol <= order_size/vol_pct
    return win_trade[frame]

In [ ]:
hist_trades = get_win_trade(tick_df, order_datetime, 4_000, 0.2)
hist_trades

Un supuesto optimista es considerar que participamos del porcentaje establecido
y que obtenemos un precio ponderado por volumen

In [ ]:
vol_weight = hist_trades.volumen/hist_trades.volumen.sum()
vol_weight

In [ ]:
vwap = (hist_trades.precio * vol_weight).sum()
vwap